<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#All-imports-necessary" data-toc-modified-id="All-imports-necessary-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>All imports necessary</a></span></li><li><span><a href="#Read-the-data" data-toc-modified-id="Read-the-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read the data</a></span></li><li><span><a href="#Naive-tag-frequency-memorization" data-toc-modified-id="Naive-tag-frequency-memorization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Naive tag frequency memorization</a></span></li><li><span><a href="#HMM" data-toc-modified-id="HMM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>HMM</a></span><ul class="toc-item"><li><span><a href="#Only-word-itself-as-a-feature" data-toc-modified-id="Only-word-itself-as-a-feature-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Only word itself as a feature</a></span></li><li><span><a href="#Two-features" data-toc-modified-id="Two-features-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Two features</a></span></li><li><span><a href="#Three-features" data-toc-modified-id="Three-features-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Three features</a></span></li><li><span><a href="#Four-features" data-toc-modified-id="Four-features-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Four features</a></span></li><li><span><a href="#Five-features" data-toc-modified-id="Five-features-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Five features</a></span></li><li><span><a href="#Six-features" data-toc-modified-id="Six-features-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Six features</a></span></li></ul></li><li><span><a href="#CRF" data-toc-modified-id="CRF-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>CRF</a></span><ul class="toc-item"><li><span><a href="#Only-word-itself-as-a-feature" data-toc-modified-id="Only-word-itself-as-a-feature-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Only word itself as a feature</a></span></li><li><span><a href="#Two-features" data-toc-modified-id="Two-features-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Two features</a></span></li><li><span><a href="#Three-features" data-toc-modified-id="Three-features-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Three features</a></span></li><li><span><a href="#Four-features" data-toc-modified-id="Four-features-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Four features</a></span></li><li><span><a href="#Five-features" data-toc-modified-id="Five-features-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Five features</a></span></li><li><span><a href="#Six-features" data-toc-modified-id="Six-features-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Six features</a></span></li></ul></li><li><span><a href="#Bi-LSTM" data-toc-modified-id="Bi-LSTM-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Bi-LSTM</a></span></li><li><span><a href="#Classification-reports" data-toc-modified-id="Classification-reports-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Classification reports</a></span><ul class="toc-item"><li><span><a href="#Naive-tag-frequency-memorization" data-toc-modified-id="Naive-tag-frequency-memorization-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Naive tag frequency memorization</a></span></li><li><span><a href="#HMM" data-toc-modified-id="HMM-7.2"><span class="toc-item-num">7.2&nbsp;&nbsp;</span>HMM</a></span></li><li><span><a href="#CRF" data-toc-modified-id="CRF-7.3"><span class="toc-item-num">7.3&nbsp;&nbsp;</span>CRF</a></span></li><li><span><a href="#Bi-LSTM" data-toc-modified-id="Bi-LSTM-7.4"><span class="toc-item-num">7.4&nbsp;&nbsp;</span>Bi-LSTM</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

# All imports necessary

In [1]:
import os
import sys
import warnings
import numpy as np
import pandas as pd

In [2]:
sys.path.append('..')
warnings.filterwarnings("ignore")

In [3]:
from source.code.utils.utils import filter_by_subcorpus
from source.code.utils.utils import get_tagged_texts_as_pd

In [4]:
from source.code.models.memorytagger import MemoryTagger
from source.code.models.bilstmtagger import BiLSTMTagger
from source.code.models.crftagger import CRFTagger
from source.code.models.hmmtagger import HMMTagger

Using TensorFlow backend.


In [5]:
from source.code.utils.preprocessing import iob3bio
from source.code.utils.preprocessing import filtrations
from source.code.utils.preprocessing import additional_features

In [6]:
from source.code.transformers.sentenceextractor import SentenceExtractor
from source.code.transformers.crftransformer import CRFTransformer

In [7]:
from seqeval.metrics import classification_report as seqeval_classification_report

In [8]:
from sklearn.model_selection import train_test_split

In [82]:
from sklearn.pipeline import Pipeline

In [9]:
import seaborn as sns

In [10]:
import matplotlib.pyplot as plt

In [11]:
%matplotlib inline

# Read the data

In [12]:
target_subcorpus_folders = filter_by_subcorpus('../data/datasets/gmb-2.2.0/', 'subcorpus: Voice of America')

In [13]:
data = get_tagged_texts_as_pd(target_subcorpus_folders, '../data/datasets/gmb-2.2.0/')

In [14]:
data = filtrations(data, with_dots=True)

In [15]:
data.ner_tag = iob3bio(data.ner_tag.values)

In [16]:
data = additional_features(df=data)

In [17]:
features = data.columns.values.tolist()

In [18]:
features.remove('ner_tag')

In [19]:
features.remove('word_net_sense_number')

In [20]:
features.remove('verb_net_roles')

In [21]:
features.remove('semantic_relation')

In [22]:
features.remove('animacy_tag')

In [23]:
features.remove('super_tag')

In [24]:
features.remove('lambda_dsr')

In [25]:
features

['token', 'pos_tag', 'lemma', 'is_title', 'contains_digits', 'word_len']

In [26]:
X, y = SentenceExtractor(features=features, target='ner_tag').fit_transform(data)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
X_train = [sentence for sentence in X_train if len(sentence) > 0]

In [29]:
y_train = [sentence.tolist() for sentence in y_train if len(sentence) > 0]

In [30]:
X_test = [sentence for sentence in X_test if len(sentence) > 0]

In [31]:
y_test = [sentence.tolist() for sentence in y_test if len(sentence) > 0]

# Naive tag frequency memorization

In [32]:
tagger = MemoryTagger()

In [33]:
tagger.fit(X_train, y_train)

MemoryTagger()

In [34]:
y_pred_memory_tagger = tagger.predict(X_test)

In [35]:
seqeval_report_memory_tagger = seqeval_classification_report(y_pred=y_pred_memory_tagger, y_true=y_test)

# HMM

[This article](https://pdfs.semanticscholar.org/9528/4b31f27b9b8901fdc18554603610ebbc2752.pdf) gives a full description of what parameters of HMM should be calculated.

From [this article](https://www.digitalvidya.com/blog/inroduction-to-hidden-markov-models-using-python/) the Viterbi algorithm implementation was taken.

## Only word itself as a feature

In [36]:
X_train_1_f = [sentence[:, features.index('lemma')] for sentence in X_train]

In [37]:
X_test_1_f = [sentence[:, features.index('lemma')] for sentence in X_test]

In [38]:
hmm_tagger = HMMTagger()

In [39]:
hmm_tagger.fit(X_train_1_f, y_train)

HMMTagger(alpha=0.01, features=None)

In [40]:
y_pred_hmm_tagger_1_f = hmm_tagger.predict(X_test_1_f)

In [69]:
seqeval_report_hmm_tagger_1_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_1_f, y_true=y_test)

## Two features

In [42]:
X_train_2_fs = [sentence[:, features.index('token'):features.index('pos_tag') + 1] for sentence in X_train]

In [43]:
X_test_2_fs = [sentence[:, features.index('token'):features.index('pos_tag') + 1] for sentence in X_test]

In [44]:
hmm_tagger = HMMTagger(features=[
    'token',
    'pos_tag'
])

In [45]:
hmm_tagger.fit(X_train_2_fs, y_train)

HMMTagger(alpha=0.01, features=['token', 'pos_tag'])

In [46]:
y_pred_hmm_tagger_2_f = hmm_tagger.predict(X_test_2_fs)

In [47]:
seqeval_report_hmm_tagger_2_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_2_f, y_true=y_test)

## Three features

In [48]:
X_train_3_fs = [sentence[:, features.index('token'):features.index('lemma') + 1] for sentence in X_train]

In [49]:
X_test_3_fs = [sentence[:, features.index('token'):features.index('lemma') + 1] for sentence in X_test]

In [50]:
hmm_tagger = HMMTagger(features=[
    'token',
    'pos_tag',
    'lemma'
])

In [51]:
hmm_tagger.fit(X_train_3_fs, y_train)

HMMTagger(alpha=0.01, features=['token', 'pos_tag', 'lemma'])

In [52]:
y_pred_hmm_tagger_3_f = hmm_tagger.predict(X_test_3_fs)

In [53]:
seqeval_report_hmm_tagger_3_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_3_f, y_true=y_test)

## Four features

In [54]:
X_train_4_fs = [sentence[:, features.index('token'):features.index('is_title') + 1] for sentence in X_train]

In [55]:
X_test_4_fs = [sentence[:, features.index('token'):features.index('is_title') + 1] for sentence in X_test]

In [56]:
hmm_tagger = HMMTagger(features=[
    'token',
    'pos_tag',
    'lemma',
    'is_title'
])

In [57]:
hmm_tagger.fit(X_train_4_fs, y_train)

HMMTagger(alpha=0.01, features=['token', 'pos_tag', 'lemma', 'is_title'])

In [58]:
y_pred_hmm_tagger_4_f = hmm_tagger.predict(X_test_4_fs)

In [59]:
seqeval_report_hmm_tagger_4_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_4_f, y_true=y_test)

## Five features

In [61]:
X_train_5_fs = [sentence[:, features.index('token'):features.index('contains_digits') + 1] for sentence in X_train]

In [62]:
X_test_5_fs = [sentence[:, features.index('token'):features.index('contains_digits') + 1] for sentence in X_test]

In [63]:
hmm_tagger = HMMTagger(features=[
    'token',
    'pos_tag',
    'lemma',
    'is_title',
    'contains_digits'
])

In [64]:
hmm_tagger.fit(X_train_5_fs, y_train)

HMMTagger(alpha=0.01,
     features=['token', 'pos_tag', 'lemma', 'is_title', 'contains_digits'])

In [65]:
y_pred_hmm_tagger_5_f = hmm_tagger.predict(X_test_5_fs)

In [66]:
seqeval_report_hmm_tagger_5_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_5_f, y_true=y_test)

## Six features

In [75]:
X_train_6_fs = [sentence[:, features.index('token'):features.index('word_len') + 1] for sentence in X_train]

In [76]:
X_test_6_fs = [sentence[:, features.index('token'):features.index('word_len') + 1] for sentence in X_test]

In [77]:
hmm_tagger = HMMTagger(features=[
    'token',
    'pos_tag',
    'lemma',
    'is_title',
    'contains_digits',
    'word_len'
])

In [78]:
hmm_tagger.fit(X_train_6_fs, y_train)

HMMTagger(alpha=0.01,
     features=['token', 'pos_tag', 'lemma', 'is_title', 'contains_digits', 'word_len'])

In [79]:
y_pred_hmm_tagger_6_f = hmm_tagger.predict(X_test_6_fs)

In [80]:
seqeval_report_hmm_tagger_6_f = seqeval_classification_report(y_pred=y_pred_hmm_tagger_6_f, y_true=y_test)

# CRF

## Only word itself as a feature

In [83]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'lemma'
    ])),
    ('fit', CRFTagger())
])

In [84]:
pipeline.fit(X_train_1_f, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['lemma'])), ('fit', CRFTagger())])

In [85]:
y_pred_crf_tagger_1_f = pipeline.predict(X_test_1_f)

In [86]:
seqeval_report_crf_tagger_1_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_1_f, y_true=y_test)

## Two features

In [88]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'token',
        'pos_tag'
    ])),
    ('fit', CRFTagger())
])

In [89]:
pipeline.fit(X_train_2_fs, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['token', 'pos_tag'])), ('fit', CRFTagger())])

In [90]:
y_pred_crf_tagger_2_f = pipeline.predict(X_test_2_fs)

In [91]:
seqeval_report_crf_tagger_2_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_2_f, y_true=y_test)

## Three features

In [92]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'token',
        'pos_tag',
        'lemma'
    ])),
    ('fit', CRFTagger())
])

In [93]:
pipeline.fit(X_train_3_fs, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['token', 'pos_tag', 'lemma'])), ('fit', CRFTagger())])

In [94]:
y_pred_crf_tagger_3_f = pipeline.predict(X_test_3_fs)

In [95]:
seqeval_report_crf_tagger_3_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_3_f, y_true=y_test)

## Four features

In [96]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'token',
        'pos_tag',
        'lemma',
        'is_title'
    ])),
    ('fit', CRFTagger())
])

In [97]:
pipeline.fit(X_train_4_fs, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['token', 'pos_tag', 'lemma', 'is_title'])), ('fit', CRFTagger())])

In [98]:
y_pred_crf_tagger_4_f = pipeline.predict(X_test_4_fs)

In [99]:
seqeval_report_crf_tagger_4_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_4_f, y_true=y_test)

## Five features

In [100]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'token',
        'pos_tag',
        'lemma',
        'is_title',
        'contains_digits'
    ])),
    ('fit', CRFTagger())
])

In [101]:
pipeline.fit(X_train_5_fs, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['token', 'pos_tag', 'lemma', 'is_title', 'contains_digits'])), ('fit', CRFTagger())])

In [102]:
y_pred_crf_tagger_5_f = pipeline.predict(X_test_5_fs)

In [103]:
seqeval_report_crf_tagger_5_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_5_f, y_true=y_test)

## Six features

In [104]:
pipeline = Pipeline([
    ('transform', CRFTransformer(features=[
        'token',
        'pos_tag',
        'lemma',
        'is_title',
        'contains_digits',
        'word_len'
    ])),
    ('fit', CRFTagger())
])

In [105]:
pipeline.fit(X_train_6_fs, y_train)

Pipeline(memory=None,
     steps=[('transform', CRFTransformer(features=['token', 'pos_tag', 'lemma', 'is_title', 'contains_digits', 'word_len'])), ('fit', CRFTagger())])

In [106]:
y_pred_crf_tagger_6_f = pipeline.predict(X_test_6_fs)

In [107]:
seqeval_report_crf_tagger_6_f = seqeval_classification_report(y_pred=y_pred_crf_tagger_6_f, y_true=y_test)

# Bi-LSTM

In [125]:
estimator = BiLSTMTagger(
    checkpoint_dir='../data/datasets/keras_model/',
    epochs=50,
    batch_size=200
)

In [126]:
estimator.fit(X_train_1_f, y_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 75)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 75, 20)            457960    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 75, 100)           28400     
_________________________________________________________________
time_distributed_3 (TimeDist (None, 75, 50)            5050      
_________________________________________________________________
crf_3 (CRF)                  (None, 75, 17)            1190      
Total params: 492,600
Trainable params: 492,600
Non-trainable params: 0
_________________________________________________________________



Epoch 00001: val_loss improved from inf to 0.12840, saving model to ../data/datasets/keras_model/model.h5



Epoch 00002: val_loss improved from 0.12840 to 0.09421, saving model to ../data/datasets/keras_model/model.h5



Epoch 00003: val_loss improved from 0.09421 to 0.06841, saving model to ../data/datasets/keras_model/model.h5



Epoch 00004: val_loss improved from 0.06841 to 0.05498, saving model to ../data/datasets/keras_model/model.h5


...


Epoch 00016: val_loss improved from 0.03018 to 0.02906, saving model to ../data/datasets/keras_model/model.h5



Epoch 00017: val_loss did not improve from 0.02906



Epoch 00018: val_loss did not improve from 0.02906



Epoch 00019: val_loss did not improve from 0.02906
Epoch 00019: early stopping


BiLSTMTagger(batch_size=200, checkpoint_dir='../data/datasets/keras_model/',
       epochs=50, max_len=75, validation_split=0.1)

In [127]:
y_pred = estimator.predict(X_test_1_f)

18724/18724 [==============================] - 33s 2ms/step


In [129]:
y_pred = [y_p[0: len(y_t)] for y_p, y_t in zip(y_pred, y_test)]

In [130]:
seqeval_report_bilstm = seqeval_classification_report(y_pred=y_pred, y_true=y_test)

# Classification reports

## Naive tag frequency memorization

In [108]:
print(seqeval_report_memory_tagger)

             precision    recall  f1-score   support

        geo       0.55      0.66      0.60     13941
        per       0.52      0.62      0.56      6861
        eve       0.47      0.17      0.25       116
        gpe       0.55      0.58      0.56      6329
        org       0.41      0.37      0.39      7629
        tim       0.76      0.71      0.73      7456
        art       0.36      0.16      0.23       146
        nat       0.38      0.13      0.19        78

avg / total       0.55      0.59      0.57     42556



## HMM

In [109]:
print(seqeval_report_hmm_tagger_1_f)

             precision    recall  f1-score   support

        geo       0.56      0.62      0.59     13941
        per       0.52      0.62      0.57      6861
        eve       0.00      0.00      0.00       116
        gpe       0.53      0.67      0.59      6329
        org       0.40      0.38      0.39      7629
        tim       0.76      0.70      0.73      7456
        art       0.41      0.12      0.18       146
        nat       0.38      0.13      0.19        78

avg / total       0.55      0.60      0.57     42556



In [110]:
print(seqeval_report_hmm_tagger_2_f)

             precision    recall  f1-score   support

        geo       0.59      0.82      0.69     13941
        per       0.53      0.64      0.58      6861
        eve       0.18      0.28      0.22       116
        gpe       0.93      0.93      0.93      6329
        org       0.44      0.54      0.49      7629
        tim       0.72      0.77      0.74      7456
        art       0.24      0.14      0.18       146
        nat       0.21      0.36      0.26        78

avg / total       0.62      0.74      0.67     42556



In [111]:
print(seqeval_report_hmm_tagger_3_f)

             precision    recall  f1-score   support

        geo       0.61      0.82      0.70     13941
        per       0.52      0.62      0.57      6861
        eve       0.10      0.41      0.16       116
        gpe       0.96      0.94      0.95      6329
        org       0.44      0.56      0.49      7629
        tim       0.54      0.83      0.65      7456
        art       0.09      0.27      0.14       146
        nat       0.23      0.54      0.32        78

avg / total       0.60      0.76      0.67     42556



In [112]:
print(seqeval_report_hmm_tagger_4_f)

             precision    recall  f1-score   support

        geo       0.61      0.83      0.70     13941
        per       0.50      0.61      0.55      6861
        eve       0.09      0.41      0.15       116
        gpe       0.95      0.94      0.94      6329
        org       0.43      0.56      0.49      7629
        tim       0.61      0.82      0.70      7456
        art       0.08      0.29      0.13       146
        nat       0.22      0.54      0.31        78

avg / total       0.61      0.75      0.67     42556



In [113]:
print(seqeval_report_hmm_tagger_5_f)

             precision    recall  f1-score   support

        geo       0.61      0.83      0.70     13941
        per       0.52      0.62      0.57      6861
        eve       0.09      0.41      0.15       116
        gpe       0.95      0.94      0.94      6329
        org       0.43      0.56      0.49      7629
        tim       0.63      0.81      0.71      7456
        art       0.09      0.29      0.13       146
        nat       0.20      0.54      0.30        78

avg / total       0.62      0.76      0.68     42556



In [114]:
print(seqeval_report_hmm_tagger_6_f)

             precision    recall  f1-score   support

        geo       0.61      0.82      0.70     13941
        per       0.51      0.61      0.56      6861
        eve       0.09      0.41      0.14       116
        gpe       0.95      0.94      0.94      6329
        org       0.42      0.55      0.47      7629
        tim       0.63      0.82      0.71      7456
        art       0.08      0.27      0.12       146
        nat       0.21      0.55      0.30        78

avg / total       0.61      0.75      0.67     42556



## CRF

In [115]:
print(seqeval_report_crf_tagger_1_f)

             precision    recall  f1-score   support

        geo       0.29      0.12      0.17     13941
        per       0.38      0.04      0.07      6861
        eve       0.00      0.00      0.00       116
        gpe       0.35      0.06      0.11      6329
        org       0.29      0.00      0.01      7629
        tim       0.55      0.06      0.11      7456
        art       0.00      0.00      0.00       146
        nat       0.00      0.00      0.00        78

avg / total       0.36      0.06      0.10     42556



In [116]:
print(seqeval_report_crf_tagger_2_f)

             precision    recall  f1-score   support

        geo       0.83      0.90      0.86     13941
        per       0.78      0.74      0.76      6861
        eve       0.50      0.25      0.33       116
        gpe       0.98      0.94      0.96      6329
        org       0.78      0.65      0.71      7629
        tim       0.90      0.79      0.84      7456
        art       0.53      0.14      0.23       146
        nat       1.00      0.26      0.41        78

avg / total       0.85      0.81      0.82     42556



In [117]:
print(seqeval_report_crf_tagger_3_f)

             precision    recall  f1-score   support

        geo       0.83      0.90      0.86     13941
        per       0.78      0.74      0.76      6861
        eve       0.46      0.22      0.30       116
        gpe       0.98      0.94      0.96      6329
        org       0.78      0.66      0.71      7629
        tim       0.90      0.79      0.84      7456
        art       0.54      0.20      0.29       146
        nat       0.54      0.24      0.34        78

avg / total       0.85      0.81      0.83     42556



In [118]:
print(seqeval_report_crf_tagger_4_f)

             precision    recall  f1-score   support

        geo       0.84      0.89      0.86     13941
        per       0.77      0.76      0.77      6861
        eve       0.42      0.22      0.29       116
        gpe       0.98      0.95      0.96      6329
        org       0.78      0.66      0.71      7629
        tim       0.90      0.79      0.84      7456
        art       0.54      0.19      0.28       146
        nat       0.54      0.24      0.34        78

avg / total       0.84      0.81      0.83     42556



In [119]:
print(seqeval_report_crf_tagger_5_f)

             precision    recall  f1-score   support

        geo       0.84      0.90      0.86     13941
        per       0.78      0.76      0.77      6861
        eve       0.42      0.22      0.29       116
        gpe       0.98      0.95      0.96      6329
        org       0.78      0.66      0.71      7629
        tim       0.89      0.79      0.84      7456
        art       0.51      0.17      0.26       146
        nat       0.57      0.26      0.35        78

avg / total       0.84      0.81      0.83     42556



In [120]:
print(seqeval_report_crf_tagger_6_f)

             precision    recall  f1-score   support

        geo       0.83      0.90      0.86     13941
        per       0.78      0.74      0.76      6861
        eve       0.44      0.22      0.30       116
        gpe       0.98      0.94      0.96      6329
        org       0.77      0.65      0.71      7629
        tim       0.90      0.79      0.84      7456
        art       0.44      0.14      0.21       146
        nat       0.57      0.26      0.35        78

avg / total       0.84      0.81      0.82     42556



## Bi-LSTM

In [131]:
print(seqeval_report_bilstm)

             precision    recall  f1-score   support

        geo       0.80      0.79      0.80     13941
        per       0.71      0.67      0.69      6861
        eve       0.70      0.12      0.21       116
        gpe       0.92      0.83      0.87      6329
        org       0.70      0.55      0.62      7629
        tim       0.85      0.83      0.84      7456
        art       0.25      0.01      0.01       146
        nat       0.67      0.08      0.14        78

avg / total       0.79      0.73      0.76     42556



# Conclusion

So here in this work several approaches were tested for NER-recognition namely:
- Naive tag frequency memorization;
- Hidden Markov model (HMM);
- Conditional Random Fields (CRF);
- Bi-Directional Long-Short Term Neural Network + CRF layer.

Every calculations were performed on the same data split with [the same](https://github.com/chakki-works/seqeval) library.

The best quality showed CRF with three features (awg precision: 0.85, awg recall: 0.81, awg f1-score: 0.83).

From HMM models the best was with five features (awg precision: 0.62, awg recall: 0.76, awg f1-score: 0.68).

Despite the tremendous simplicity of naive approach it was able to show relatively good performance (awg precision: 0.55, awg recall: 0.59, awg f1-score: 0.57).

Fits quite well as a baseline.